# Student Module Documentation

This notebook documents the `Student`, `Enrollment`, `Guardian`, `AcademicRecord`, `Document`, and `EnrollmentHistory` models and their serializers in the `apps.student` module.

---

## Models Overview

### 1. BaseModel (Base)
Abstract base class providing automatic UUIDs and timestamps for all student models.

| Field | Type | Description |
|-------|------|-------------|
| `ukid` | UUIDField | Unique identifier (auto-generated, indexed) |
| `created_at` | DateTimeField | Automatically set when record is created |
| `created_by` | IntegerField | User ID who created the record (nullable) |
| `updated_at` | DateTimeField | Automatically updated on every save |
| `updated_by` | IntegerField | User ID who last updated the record (nullable) |

---

### 2. Student
Represents a student enrolled in the institution.

| Field | Type | Constraints | Description |
|-------|------|-------------|-------------|
| `id` | AutoField | Primary Key | Auto-generated unique identifier |
| `ukid` | UUIDField | Unique, Indexed | Inherited from BaseModel |
| `registration_number` | BigIntegerField | Unique, Required | Student registration number |
| `roll_number` | CharField | max_length=20, Unique, Required | Student roll number |
| `first_name` | CharField | max_length=50, Required | First name |
| `middle_name` | CharField | max_length=50, Optional | Middle name |
| `last_name` | CharField | max_length=50, Required | Last name |
| `date_of_birth` | DateField | Required | Date of birth |
| `gender` | CharField | max_length=10, Required | Gender |
| `email` | EmailField | Unique, Required | Email address |
| `phone_number` | CharField | max_length=15, Required | Phone number |
| `address` | TextField | Required | Full address |
| `city` | CharField | max_length=50, Required | City |
| `state` | CharField | max_length=50, Required | State/Province |
| `postal_code` | CharField | max_length=10, Required | Postal/ZIP code |
| `country` | CharField | max_length=50, Required | Country |
| `enrollment_date` | DateField | Required | Date of enrollment |
| `is_active` | BooleanField | default=True | Active status (soft delete) |
| `deleted_at` | DateTimeField | Optional | When student was deleted |
| `deleted_by` | CharField | max_length=100, Optional | Who deleted the student |
| `created_at` | DateTimeField | auto_now_add | Inherited from BaseModel |
| `created_by` | IntegerField | Optional | Inherited from BaseModel |
| `updated_at` | DateTimeField | auto_now | Inherited from BaseModel |
| `updated_by` | IntegerField | Optional | Inherited from BaseModel |

**Relationships:**
- Has many `Enrollment` via `enrollments` related_name
- Has many `Guardian` via `guardians` related_name (ManyToMany)
- Has many `AcademicRecord` via `academic_records` related_name
- Has many `Document` via `documents` related_name
- Has many `EnrollmentHistory` via `enrollment_histories` related_name

**Ordering:** `['first_name', 'last_name']`

**Indexes:**
- `['email']`
- `['registration_number']`
- `['is_active', 'enrollment_date']`

**Validation Rules:**
- Student must be at least 3 years old
- Age cannot exceed 100 years
- `enrollment_date` cannot be in the future

---

### 3. Enrollment
Represents a student's enrollment in a program.

| Field | Type | Constraints | Description |
|-------|------|-------------|-------------|
| `id` | AutoField | Primary Key | Auto-generated unique identifier |
| `ukid` | UUIDField | Unique, Indexed | Inherited from BaseModel |
| `student` | ForeignKey | Required, CASCADE | Student enrolled |
| `program` | CharField | max_length=100, Required | Program name |
| `batch` | CharField | max_length=6, Required | Batch year (e.g., "2024") |
| `section` | CharField | max_length=10, Required | Section (e.g., "A") |
| `semester` | CharField | max_length=20, Required | Semester |
| `enrollment_date` | DateField | Required | Date of enrollment |
| `status` | CharField | max_length=20, default='enrolled' | Enrollment status (see choices) |
| `created_at` | DateTimeField | auto_now_add | Inherited from BaseModel |
| `created_by` | IntegerField | Optional | Inherited from BaseModel |
| `updated_at` | DateTimeField | auto_now | Inherited from BaseModel |
| `updated_by` | IntegerField | Optional | Inherited from BaseModel |

**Status Choices:**
| Value | Constant | Label |
|-------|----------|-------|
| `enrolled` | `ENROLLED` | Enrolled |
| `completed` | `COMPLETED` | Completed |
| `dropped` | `DROPPED` | Dropped |
| `suspended` | `SUSPENDED` | Suspended |

**Unique Together:** `['student', 'program', 'semester']`

**Ordering:** `['-enrollment_date']`

**Indexes:**
- `['program', 'batch']`
- `['student', 'status']`

---

### 4. Guardian
Represents a guardian/parent of one or more students.

| Field | Type | Constraints | Description |
|-------|------|-------------|-------------|
| `id` | AutoField | Primary Key | Auto-generated unique identifier |
| `ukid` | UUIDField | Unique, Indexed | Inherited from BaseModel |
| `student` | ManyToManyField | Required | Students under this guardian |
| `first_name` | CharField | max_length=50, Required | First name |
| `last_name` | CharField | max_length=50, Required | Last name |
| `relationship` | CharField | max_length=50, Required | Relationship (e.g., "Mother", "Father") |
| `phone_number` | CharField | max_length=15, Required | Phone number |
| `email` | EmailField | Required | Email address |
| `address` | TextField | Required | Full address |
| `created_at` | DateTimeField | auto_now_add | Inherited from BaseModel |
| `created_by` | IntegerField | Optional | Inherited from BaseModel |
| `updated_at` | DateTimeField | auto_now | Inherited from BaseModel |
| `updated_by` | IntegerField | Optional | Inherited from BaseModel |

**Ordering:** `['first_name', 'last_name']`

**Indexes:**
- `['email']`
- `['phone_number']`

---

### 5. AcademicRecord
Represents a student's academic performance for a semester.

| Field | Type | Constraints | Description |
|-------|------|-------------|-------------|
| `id` | AutoField | Primary Key | Auto-generated unique identifier |
| `ukid` | UUIDField | Unique, Indexed | Inherited from BaseModel |
| `student` | ForeignKey | Required, CASCADE | Student record belongs to |
| `semester` | CharField | max_length=20, Required | Semester name |
| `gpa` | DecimalField | max_digits=4, decimal_places=2, Required | Grade Point Average |
| `total_credits` | IntegerField | Required | Total credits earned |
| `remarks` | TextField | Optional | Additional remarks |
| `created_at` | DateTimeField | auto_now_add | Inherited from BaseModel |
| `created_by` | IntegerField | Optional | Inherited from BaseModel |
| `updated_at` | DateTimeField | auto_now | Inherited from BaseModel |
| `updated_by` | IntegerField | Optional | Inherited from BaseModel |

**Ordering:** `['-semester']`

**Validation Rules:**
- GPA must be between 0.00 and 4.00
- `total_credits` cannot be negative

---

### 6. Document
Represents a document uploaded for a student.

| Field | Type | Constraints | Description |
|-------|------|-------------|-------------|
| `id` | AutoField | Primary Key | Auto-generated unique identifier |
| `ukid` | UUIDField | Unique, Indexed | Inherited from BaseModel |
| `student` | ForeignKey | Required, CASCADE | Student document belongs to |
| `document_type` | CharField | max_length=255, Required | Type of document (see choices) |
| `file` | FileField | upload_to='student_documents/', Required | Uploaded file |
| `is_verified` | BooleanField | default=False | Verification status |
| `verified_by` | CharField | max_length=100, Optional | Who verified the document |
| `verified_at` | DateTimeField | Optional | When document was verified |
| `created_at` | DateTimeField | auto_now_add | Inherited from BaseModel |
| `created_by` | IntegerField | Optional | Inherited from BaseModel |
| `updated_at` | DateTimeField | auto_now | Inherited from BaseModel |
| `updated_by` | IntegerField | Optional | Inherited from BaseModel |

**Document Type Choices:**
| Value | Constant | Label |
|-------|----------|-------|
| `id_proof` | `ID_PROOF` | ID Proof |
| `birth_cert` | `BIRTH_CERTIFICATE` | Birth Certificate |
| `transcript` | `ACADEMIC_TRANSCRIPT` | Academic Transcript |
| `photo` | `PHOTOGRAPH` | Photograph |
| `medical` | `MEDICAL_CERTIFICATE` | Medical Certificate |
| `transfer` | `TRANSFER_CERTIFICATE` | Transfer Certificate |
| `other` | `OTHER` | Other |

**Ordering:** `['-created_at']`

---

### 7. EnrollmentHistory
Tracks changes in enrollment status over time.

| Field | Type | Constraints | Description |
|-------|------|-------------|-------------|
| `id` | AutoField | Primary Key | Auto-generated unique identifier |
| `ukid` | UUIDField | Unique, Indexed | Inherited from BaseModel |
| `enrollment` | ForeignKey | Required, CASCADE | Enrollment this history belongs to |
| `student` | ForeignKey | Required, CASCADE | Student |
| `semester` | CharField | max_length=20, Required | Semester |
| `previous_status` | CharField | max_length=20, Required | Previous enrollment status |
| `new_status` | CharField | max_length=20, Required | New enrollment status |
| `created_at` | DateTimeField | auto_now_add | Inherited from BaseModel |
| `created_by` | IntegerField | Optional | Inherited from BaseModel |
| `updated_at` | DateTimeField | auto_now | Inherited from BaseModel |
| `updated_by` | IntegerField | Optional | Inherited from BaseModel |

**Ordering:** `['-created_at']`

**Indexes:**
- `['student', 'created_at']`
- `['enrollment']`

---

## API Serializers

The Student module follows a three-serializer pattern for each model:

### Student Serializers

**StudentCreateSerializer** - For POST operations
- Fields: `registration_number`, `roll_number`, `first_name`, `middle_name`, `last_name`, `date_of_birth`, `gender`, `email`, `phone_number`, `address`, `city`, `state`, `postal_code`, `country`, `enrollment_date`
- Validation: Age 3-100 years, enrollment date not in future

**StudentUpdateSerializer** - For PUT/PATCH operations
- Fields: All Create fields except `registration_number` plus `is_active`, `updated_by` (all optional)
- Validation: Same age validation as Create

**StudentResponseSerializer** - For GET operations
- Fields: All model fields plus computed `full_name` and `enrollment_count`
- Computed: `full_name` (combines name parts), `enrollment_count`
- Read-only: `ukid`, `registration_number`, `created_at`, `updated_at`

### Enrollment Serializers

**EnrollmentCreateSerializer** - For POST operations
- Fields: `student`, `program`, `batch`, `section`, `semester`, `enrollment_date`

**EnrollmentUpdateSerializer** - For PUT/PATCH operations
- Fields: `program`, `batch`, `section`, `semester`, `status`, `updated_by` (all optional)

**EnrollmentResponseSerializer** - For GET operations
- Fields: All model fields with nested `student` object
- Nested: Full Student object
- Computed: `status_display`
- Read-only: `ukid`, `created_at`, `updated_at`

### Guardian Serializers

**GuardianCreateSerializer** - For POST operations
- Fields: `student`, `first_name`, `last_name`, `relationship`, `phone_number`, `email`, `address`

**GuardianUpdateSerializer** - For PUT/PATCH operations
- Fields: `first_name`, `last_name`, `relationship`, `phone_number`, `email`, `address`, `updated_by` (all optional)

**GuardianResponseSerializer** - For GET operations
- Fields: All model fields
- Computed: `full_name`, `student_count`
- Read-only: `ukid`, `created_at`, `updated_at`

### AcademicRecord Serializers

**AcademicRecordCreateSerializer** - For POST operations
- Fields: `student`, `semester`, `gpa`, `total_credits`, `remarks`
- Validation: GPA 0.00-4.00, credits >= 0

**AcademicRecordUpdateSerializer** - For PUT/PATCH operations
- Fields: `semester`, `gpa`, `total_credits`, `remarks`, `updated_by` (all optional)
- Validation: Same as Create

**AcademicRecordResponseSerializer** - For GET operations
- Fields: All model fields with nested `student` object
- Nested: Full Student object
- Read-only: `ukid`, `created_at`, `updated_at`

### Document Serializers

**DocumentCreateSerializer** - For POST operations
- Fields: `student`, `document_type`, `file`

**DocumentUpdateSerializer** - For PUT/PATCH operations
- Fields: `document_type`, `file`, `is_verified`, `updated_by` (all optional)
- Auto-set: When `is_verified` becomes true, sets `verified_at` and `verified_by`

**DocumentResponseSerializer** - For GET operations
- Fields: All model fields with nested `student` object
- Nested: Full Student object
- Computed: `document_type_display`
- Read-only: `ukid`, `verified_by`, `verified_at`, `created_at`, `updated_at`

### EnrollmentHistory Serializers

**EnrollmentHistoryCreateSerializer** - For POST operations
- Fields: `enrollment`, `student`, `semester`, `previous_status`, `new_status`

**EnrollmentHistoryUpdateSerializer** - For PUT/PATCH operations
- Fields: `semester`, `previous_status`, `new_status`, `updated_by` (all optional)

**EnrollmentHistoryResponseSerializer** - For GET operations
- Fields: All model fields with nested `student` and `enrollment` objects
- Nested: Full Student and Enrollment objects
- Computed: `previous_status_display`, `new_status_display`
- Read-only: `ukid`, `created_at`, `updated_at`

---

## Usage Examples

```python
from apps.student.models import Student, Enrollment, Guardian, AcademicRecord
from datetime import date

# Create a Student
student = Student.objects.create(
    registration_number=202400001,
    roll_number="CS2024001",
    first_name="John",
    middle_name="Robert",
    last_name="Doe",
    date_of_birth=date(2005, 1, 15),
    gender="Male",
    email="john.doe@example.com",
    phone_number="+1234567890",
    address="123 Main St",
    city="New York",
    state="NY",
    postal_code="10001",
    country="USA",
    enrollment_date=date(2024, 9, 1)
)

# Create an Enrollment
enrollment = Enrollment.objects.create(
    student=student,
    program="Computer Science",
    batch="2024",
    section="A",
    semester="Fall 2024",
    enrollment_date=date(2024, 9, 1)
)

# Create a Guardian
guardian = Guardian.objects.create(
    first_name="Jane",
    last_name="Doe",
    relationship="Mother",
    phone_number="+1234567891",
    email="jane.doe@example.com",
    address="123 Main St"
)
guardian.student.add(student)

# Create an Academic Record
record = AcademicRecord.objects.create(
    student=student,
    semester="Fall 2024",
    gpa=3.75,
    total_credits=18,
    remarks="Excellent performance"
)

# Soft delete a student
student.is_active = False
student.save()
```

---

## Validation Rules

### Student Validation
1. Age must be between 3 and 100 years
2. `enrollment_date` cannot be in the future

### AcademicRecord Validation
1. GPA must be between 0.00 and 4.00
2. `total_credits` cannot be negative

### Enrollment Constraints
1. Unique together: `student`, `program`, `semester`
2. Cannot have duplicate enrollments for same program and semester

---

## API Endpoints

Base URL: `/api/student/`

- `/students/` - Student CRUD + `/active/`, `/inactive/` actions
- `/enrollments/` - Enrollment CRUD
- `/guardians/` - Guardian CRUD
- `/academic-records/` - Academic Record CRUD
- `/documents/` - Document CRUD
- `/enrollment-history/` - Enrollment History CRUD

All endpoints support standard REST operations:
- GET (list/retrieve)
- POST (create)
- PUT/PATCH (update)
- DELETE (delete/soft-delete)

---

## Special Features

### Soft Delete (Student)
- DELETE operation sets `is_active=False` instead of removing record
- Auto-sets `deleted_at` timestamp and `deleted_by` user

### Document Verification
- When `is_verified` changes to `True`, automatically sets:
  - `verified_at` = current timestamp
  - `verified_by` = current user username

### Computed Fields
- Student: `full_name`, `enrollment_count`
- Guardian: `full_name`, `student_count`
- All status fields: `{field}_display` for human-readable values

### Nested Responses
- Response serializers include full related objects
- Enrollment includes full Student object
- AcademicRecord includes full Student object
- Document includes full Student object
- EnrollmentHistory includes full Student and Enrollment objects